#### 📥Importaciones 

In [8]:
# Importación del módulo pickle para la serialización de objetos
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import os
import pyarrow.parquet as pq  # Importamos Parquet
# Importación del módulo warnings para controlar advertencias
import warnings
# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd
# Importación de funciones desde data_utils
from data_utils import analisis_sentimiento, ej_review_sentimiento
# Configuración de la omisión de advertencias
warnings.filterwarnings("ignore")
# Carga de la extensión autoreload y configuración para recargar automáticamente módulos
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



### 📊 Análisis de Sentimientos

**'sentiment_analysis'** reemplaza a **'reviews_review'** : se crea esta nueva columna para realizar un análisis de sentimientos en los comentarios de los usuarios.



📦 Extracción de los conjuntos de datos
* ✍️ **Extraemos df_user_reviews:** usuarios que realizaron reseñas de los juegos que consumen.


In [9]:
df_user_reviews = pd.read_parquet('../0 Dataset/1.3_user_review_LISTO.parquet')

**Utilizamos la biblioteca TextBlob** , que es una herramienta de procesamiento de lenguaje natural (NLP) en Python en esta columna y aplico un análisis de sentimiento básico.

La escala utilizada es la siguiente:

* 👎 **0** si el sentimiento es **malo.**
* 😐 **1** si el sentimiento es **neutral o no hay revisión.**
* 👍 **2** si el sentimiento es **positivo.**

Esta metodología asigna un valor numérico a cada texto, en este caso, a los comentarios de los usuarios en relación con un juego específico, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo.

El análisis se basa en la polaridad calculada por TextBlob, donde se considera que las polaridades negativas están por debajo de -0.2, las positivas por encima de 0.2 y las neutrales entre estos valores.


In [10]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['reviews_review'].apply(analisis_sentimiento)
df_user_reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,1


In [11]:
#Reviso algunos ejemplos para cada una de las clases de sentimiento.
ej_review_sentimiento(df_user_reviews['reviews_review'], df_user_reviews['sentiment_analysis'])

En la sección de evaluación de sentimientos, se presentan los siguientes ejemplos de reseñas para 0:
Reseña: 1: This game is Marvellous.
Reseña: 2: Killed the Emperor, nobody cared and got away with it. Accidentally killed a chicken and everybody decided to gang up on me. 10/10
Reseña: 3: This Game Doesn't Work


En la sección de evaluación de sentimientos, se presentan los siguientes ejemplos de reseñas para 1:
Reseña: 1: Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.
Reseña: 2: I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what yo

🗑️ Al final del proceso, la columna 'reviews_review' se elimina del conjunto de datos. 


In [12]:
df_user_reviews = df_user_reviews.drop(columns=['reviews_review'])

#Actualizamos nuestro parquet con los cambios nuevos:
# Convert DataFrame to PyArrow Table
table = pa.Table.from_pandas(df_user_reviews)

ruta_parquet = os.path.join('..', '0 Dataset', '2.1_user_review_LISTO.parquet')
df_user_reviews.to_parquet(ruta_parquet)
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             57397 non-null  object
 1   user_url            57397 non-null  object
 2   reviews_item_id     57397 non-null  object
 3   reviews_helpful     57397 non-null  object
 4   reviews_recommend   57397 non-null  bool  
 5   sentiment_analysis  57397 non-null  int64 
dtypes: bool(1), int64(1), object(4)
memory usage: 2.7+ MB
